In [17]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import root_mean_squared_error

import mlflow

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import xgboost as xgb

import pickle

In [10]:
pd.options.mode.chained_assignment = None  # default='warn'

In [11]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db
mlflow.set_tracking_uri(uri="sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1721809120839, experiment_id='1', last_update_time=1721809120839, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Data Processing

In [12]:
def import_data(url):
  return pd.read_parquet(url)

In [13]:
def process_data(df):
  df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).apply(lambda x: x.total_seconds() / 60)
  df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
  df['PU_DO'] = df.PULocationID + '_' + df.DOLocationID
  categorical.append('PU_DO')
  
  numerical = ['trip_distance', 'duration']

  return df[categorical + numerical]

In [14]:
def transform_data(df, dv=None):
  # predictors = ['PULocationID', 'DOLocationID']
  predictors = ['PU_DO', 'trip_distance']
  target = 'duration'

  df_dicts = df[predictors].to_dict(orient='records')

  if dv:
    X = dv.transform(df_dicts)
  else:
    dv = DictVectorizer()
    X = dv.fit_transform(df_dicts)
  
  y = df[target].values

  return X, y, dv

In [15]:
def compute_error(X, y, model):
  preds = model.predict(X)
  error = root_mean_squared_error(preds, y)

  return error

In [16]:
train_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet"
val_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet"

train_df = import_data(train_url)
train_df = process_data(train_df)
X_train, y_train, dv = transform_data(train_df)

val_df = import_data(val_url)
val_df = process_data(val_df)
X_val, y_val, _ = transform_data(val_df, dv)

## Manual Logging

In [9]:
alpha = .01
model = Lasso(alpha=alpha)


model.fit(X_train, y_train)
rmse = compute_error(X_val, y_val, model)

with mlflow.start_run():
  mlflow.set_tag("developer", "Bastien Winant")

  mlflow.log_params({
    "train_data": train_url,
    "val_data": val_url,
    "alpha": alpha
  })


  mlflow.log_metric("rmse", rmse)

## Hyperparameter Tuning

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [11]:
def objective(params):
  booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
  )

  preds = booster.predict(valid)
  rmse = root_mean_squared_error(preds, y_val)
  
  with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)
    mlflow.log_metric("rmse", rmse)
  
  return {"loss": rmse, "status": STATUS_OK}

In [12]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'reg:linear',
  'seed': 42
}

best_result = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=50,
  trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:18:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.95707                          
[1]	validation-rmse:9.96230                           
[2]	validation-rmse:9.18036                           
[3]	validation-rmse:8.58192                           
[4]	validation-rmse:8.09857                           
[5]	validation-rmse:7.74662                           
[6]	validation-rmse:7.46771                           
[7]	validation-rmse:7.25924                           
[8]	validation-rmse:7.10225                           
[9]	validation-rmse:6.98494                           
[10]	validation-rmse:6.89293                          
[11]	validation-rmse:6.82250                          
[12]	validation-rmse:6.76023                          
[13]	validation-rmse:6.71796                          
[14]	validation-rmse:6.67723                          
[15]	validation-rmse:6.65204                          
[16]	validation-rmse:6.63042                          
[17]	validation-rmse:6.60816                          
[18]	valid

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:20:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:6.81113                                                    
[1]	validation-rmse:6.71755                                                    
[2]	validation-rmse:6.69675                                                    
[3]	validation-rmse:6.68679                                                    
[4]	validation-rmse:6.67982                                                    
[5]	validation-rmse:6.66945                                                    
[6]	validation-rmse:6.66564                                                    
[7]	validation-rmse:6.66124                                                    
[8]	validation-rmse:6.65773                                                    
[9]	validation-rmse:6.65556                                                    
[10]	validation-rmse:6.65565                                                   
[11]	validation-rmse:6.65404                                                   
[12]	validation-rmse:6.65177            

...

[0]	validation-rmse:10.98429                                                    
[1]	validation-rmse:10.00167                                                    
[2]	validation-rmse:9.23028                                                     
[3]	validation-rmse:8.62000                                                     
[4]	validation-rmse:8.14962                                                     
[5]	validation-rmse:7.78706                                                     
[6]	validation-rmse:7.50252                                                     
[7]	validation-rmse:7.29138                                                     
[8]	validation-rmse:7.13215                                                     
[9]	validation-rmse:7.00355                                                     
[10]	validation-rmse:6.91387                                                    
[11]	validation-rmse:6.83377                                                    
[12]	validation-rmse:6.77174

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:16:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.47184                                                    
[1]	validation-rmse:9.24432                                                     
[2]	validation-rmse:8.39171                                                     
[3]	validation-rmse:7.81120                                                     
[4]	validation-rmse:7.42169                                                     
[5]	validation-rmse:7.15281                                                     
[6]	validation-rmse:6.97936                                                     
[7]	validation-rmse:6.86273                                                     
[8]	validation-rmse:6.77997                                                     
[9]	validation-rmse:6.71580                                                     
[10]	validation-rmse:6.67380                                                    
[11]	validation-rmse:6.63896                                                    
[12]	validation-rmse:6.61795

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:17:07] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.99704                                                     
[1]	validation-rmse:7.56930                                                     
[2]	validation-rmse:6.97964                                                     
[3]	validation-rmse:6.73805                                                     
[4]	validation-rmse:6.62770                                                     
[5]	validation-rmse:6.57419                                                     
[6]	validation-rmse:6.54093                                                     
[7]	validation-rmse:6.52603                                                     
[8]	validation-rmse:6.50884                                                     
[9]	validation-rmse:6.49772                                                     
[10]	validation-rmse:6.49335                                                    
[11]	validation-rmse:6.48879                                                    
[12]	validation-rmse:6.48318

## Autologging

In [13]:
# retrieve the best-performing params from mlflow
best_params = {
  "learning_rate": 0.21668937995954535,
  "max_depth": 18,
  "min_child_weight": 1.0625241915799823,
  "objective": "reg:linear",
  "reg_alpha": 0.014108586386588398,
  "reg_lambda": 0.007062245165893128,
  "seed": 42
}

In [14]:
mlflow.xgboost.autolog()

# train model with best params
booster = xgb.train(
  params=best_params,
  dtrain=train,
  num_boost_round=1000,
  evals=[(valid, "validation")],
  early_stopping_rounds=50
)

2024/07/24 11:41:18 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2024/07/24 11:41:19 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '376b419a02484c1e9476f7984b11b842', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:41:29] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.53069
[1]	validation-rmse:9.32437
[2]	validation-rmse:8.47536
[3]	validation-rmse:7.89386
[4]	validation-rmse:7.49437
[5]	validation-rmse:7.22167
[6]	validation-rmse:7.03833
[7]	validation-rmse:6.91420
[8]	validation-rmse:6.82476
[9]	validation-rmse:6.76573
[10]	validation-rmse:6.72255
[11]	validation-rmse:6.69032
[12]	validation-rmse:6.66784
[13]	validation-rmse:6.64784
[14]	validation-rmse:6.63216
[15]	validation-rmse:6.62061
[16]	validation-rmse:6.61052
[17]	validation-rmse:6.60383
[18]	validation-rmse:6.59924
[19]	validation-rmse:6.59532
[20]	validation-rmse:6.59425
[21]	validation-rmse:6.59283
[22]	validation-rmse:6.59114
[23]	validation-rmse:6.58899
[24]	validation-rmse:6.58693
[25]	validation-rmse:6.58533
[26]	validation-rmse:6.58384
[27]	validation-rmse:6.58115
[28]	validation-rmse:6.57882
[29]	validation-rmse:6.57593
[30]	validation-rmse:6.57317
[31]	validation-rmse:6.57063
[32]	validation-rmse:6.56899
[33]	validation-rmse:6.56818
[34]	validation-rmse:6.

2024/07/24 11:43:00 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/07/24 11:43:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:43:00] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


## Manual Model Management

In [18]:
alpha = .1
lasso = Lasso(alpha)
lasso.fit(X=X_train, y=y_train)
preds = lasso.predict(X_val)
rmse = root_mean_squared_error(preds, y_val)

with open("models/lin_reg.bin", "bw") as f_out:
  pickle.dump((dv, lasso), f_out)

with mlflow.start_run():
  mlflow.set_tag("developer", "Bastien Winant")

  mlflow.log_params({
    "train_data": train_url,
    "val_data": val_url,
    "aplha": alpha
  })

  mlflow.log_metric("rmse", rmse)

  # save model as an artifact
  mlflow.log_artifact("models/lin_reg.bin", artifact_path="models_pickle")

## Automated Model Management

In [21]:
# retrieve the best-performing params from mlflow
best_params = {
  "learning_rate": 0.21668937995954535,
  "max_depth": 18,
  "min_child_weight": 1.0625241915799823,
  "objective": "reg:linear",
  "reg_alpha": 0.014108586386588398,
  "reg_lambda": 0.007062245165893128,
  "seed": 42
}

booster = xgb.train(
  params=best_params,
  dtrain=train,
  num_boost_round=1000,
  evals=[(valid, "validation")],
  early_stopping_rounds=50
)

preds = booster.predict(valid)
rmse = root_mean_squared_error(y_val, preds)

with open("models/preprocessor.b", "wb") as f_out:
  pickle.dump(dv, f_out)

with mlflow.start_run():
  mlflow.log_params(best_params)
  mlflow.log_metric("rmse", rmse)

  # log the model
  mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
  mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:10:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.53069
[1]	validation-rmse:9.32437
[2]	validation-rmse:8.47536
[3]	validation-rmse:7.89386
[4]	validation-rmse:7.49437
[5]	validation-rmse:7.22167
[6]	validation-rmse:7.03833
[7]	validation-rmse:6.91420
[8]	validation-rmse:6.82476
[9]	validation-rmse:6.76573
[10]	validation-rmse:6.72255
[11]	validation-rmse:6.69032
[12]	validation-rmse:6.66784
[13]	validation-rmse:6.64784
[14]	validation-rmse:6.63216
[15]	validation-rmse:6.62061
[16]	validation-rmse:6.61052
[17]	validation-rmse:6.60383
[18]	validation-rmse:6.59924
[19]	validation-rmse:6.59532
[20]	validation-rmse:6.59425
[21]	validation-rmse:6.59283
[22]	validation-rmse:6.59114
[23]	validation-rmse:6.58899
[24]	validation-rmse:6.58693
[25]	validation-rmse:6.58533
[26]	validation-rmse:6.58384
[27]	validation-rmse:6.58115
[28]	validation-rmse:6.57882
[29]	validation-rmse:6.57593
[30]	validation-rmse:6.57317
[31]	validation-rmse:6.57063
[32]	validation-rmse:6.56899
[33]	validation-rmse:6.56818
[34]	validation-rmse:6.

/Users/bastienwinant/Desktop/Projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:11:44] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


## Retrieve model and make predictions

In [23]:
logged_model = 'runs:/4cffa5e1dfc542ceb451d95d681c51a6/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [27]:
# Predict on a Pandas DataFrame.
loaded_model.predict(X_val)

array([14.457083,  7.050064, 15.660235, ..., 13.59472 ,  6.457606,
        8.311016], dtype=float32)

In [24]:
xgboost_model.predict(valid)

array([14.457083,  7.050064, 15.660235, ..., 13.59472 ,  6.457606,
        8.311016], dtype=float32)